# Generative parameters

In [ ]:
# import wandb
# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="spectrum",  
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.001,
#     "architecture": "SpectrumEncoder",
#     "dataset": "Spectrum Dataset",
#     "epochs": 50,
#     }
# )

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


dataset = np.loadtxt('spectrum_dataset.3.csv', delimiter='\t')
X = dataset[:,0:64]
y = dataset[:,64:68]

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

import torch
import torch.nn as nn
import torch.nn.functional as F
    
class SpectrumEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(64, 512)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(256, 4)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x
    

model = SpectrumEncoder()
print(model)
        
# train the model
loss_fn   = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 100 
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % batch_size == 0:
        print(f'Epoch : {epoch}, Training: Loss: {loss}')
        # wandb.log({'epoch': epoch, 'loss': loss})



# make class predictions with the model
predictions = model(X) #(model(X) > 0.5).float()
for i in range(10):
    # print(type(predictions[i]), type(y[i]))
    a = predictions[i]
    b = y[i]
    distance = torch.sqrt(torch.sum(torch.pow(torch.subtract(a, b), 2), dim=0))
    #print('%s => %s' % (y[i].tolist(), predictions[i].tolist()))
    print(f"{i} distance: {distance} {a}")

print("done")



In [ ]:
# make class predictions with the model
predictions = model(X) #(model(X) > 0.5).float()
for i in range(10):
    # print(type(predictions[i]), type(y[i]))
    a = predictions[i]
    b = y[i]
    distance = torch.sqrt(torch.sum(torch.pow(torch.subtract(a, b), 2), dim=0))
    #print('%s => %s' % (y[i].tolist(), predictions[i].tolist()))
    print(f"{i} distance: {distance} {a} {b}")

print("done")

In [ ]:
torch.save(model.state_dict(), 'spectrum.model')

# import wandb
# import random
# # Start a new W&B run
# with wandb.init(project="spectrum") as run:
#     # Simulate logging model metrics
#     run.log({"acc": random.random()})
#     # Save the dummy model to W&B
#     best_model = wandb.Artifact(f"model_{run.id}", type="model")
#     best_model.add_file("spectrum.h5")
#     run.log_artifact(best_model)
#     # Link the model to the Model Registry
#     run.link_artifact(best_model, "model-registry/My Registered Model")
#     run.finish()

In [ ]:
#from autoPyTorch import AutoNetClassification

In [ ]:
#torch.onnx.export(model, X, 'spectrum.onnx', input_names=["features"], output_names=["logits"])

In [ ]:
#torch.save(model.state_dict(), 'spectrum.model')